### 【マルチスイープ結果の抽出】

In [83]:
import pandas as pd
import numpy as np

# ---------------------------------------------------
# ----入力パラメータ-----------------------------------
# ---------------------------------------------------
# fpeak: C6 = 6.0 pF, L2 = 27.0 nH, C5 = 27.0 pF
# gain:  C6 = 6.0 pF, C12 = 2.0 nH, L2 = 27.0 nH
# image: C4 = 2.0 pF, L2 = 27.0 nH, C6 = 6.0 pF

# 各定数のステップ数
step1 = 6
step2 = 6
step3 = 6

VTUNE12 = [[1.0, 1.1, 1.2, 1.3, 1.4],
    [2.2, 2.3, 2.4, 2.5, 2.6],
    [3.2, 3.3, 3.4, 3.5, 3.6]]

# ピーク周波数の寄与上位
fpeak = [[5.7,5.82,5.94,6.06,6.18,6.3],
    [26.7,26.82,26.94,27.06,27.18,27.3],
    [26.7,26.82,26.94,27.06,27.18,27.3]]

# ゲインの寄与上位
gain = [[5.7,5.82,5.94,6.06,6.18,6.3],
    [1.7,1.82,1.94,2.06,2.18,2.3],
    [26.7,26.82,26.94,27.06,27.18,27.3]]

# イメージの寄与上位
image = [[1.7,1.82,1.94,2.06,2.18,2.3],
    [26.7,26.82,26.94,27.06,27.18,27.3],
    [5.7,5.82,5.94,6.06,6.18,6.3]]

########################################################
# 周波数ピーク、ゲイン、イメージ抑圧に対して
# 各一つずつをコメント外して実行すること
########################################################

# (1) ファイルパスの編集
prefix = './data/02_multi/'
fname = prefix + "fpeak_fL.csv" # 入力ファイル名
fname_out = prefix + "output/" + "fpeak_fL_out.csv" # 出力ファイル名

# fname = prefix + "gain_fH.csv"
# fname_out = prefix + "output/" + "gain_fH_out.csv"

# fname = prefix + "img_fH.csv"
# fname_out = prefix + "output/" + "img_fH_out.csv"

# (2) 目的変数の選択(fpeak, gain, image)
val = fpeak
# val = gain
# val = image

# (3) 調整値の選択(0:fL, 1:fM, 2:fH)
Tf = VTUNE12[0] # fL
# Tf = VTUNE12[1] # fM
# Tf = VTUNE12[2] # fH
########################################################

In [84]:
# ヘッダを加える
df = pd.read_csv(fname, header = None, names = ["ID","Val1","Val2","Val3","VTUNE12","Freq","S21"])

# 先頭行と末尾行を削除する
df = df.drop(index=0)
df = df.drop(index=df.shape[0])

# IDは不要のため抹消する
df = df.drop(columns="ID")

# データフレーム結合
df_all = pd.DataFrame(columns=["Val1","Val2","Val3","VTUNE12","fpeak","gain","fimg","img"])

In [85]:
# T調整値を選択
for i, val0 in enumerate(val[0]):

    # 1つ目のパラメータ
    for j, val1 in enumerate(val[1]):
        # 2つ目のパラメータ
        for k, val2 in enumerate(val[2]):
            # 3つ目のパラメータ
            for l, T in enumerate(Tf):

                # ループで使用するデータフレームを抽出
                df_tmp = df.loc[df["Val1"]==val0]
                df_tmp = df_tmp.loc[df_tmp["Val2"]==val1]
                df_tmp = df_tmp.loc[df_tmp["Val3"]==val2]
                df_tmp = df_tmp.loc[df_tmp["VTUNE12"]==T]

                # ピーク値を取る
                tmp = df_tmp[df_tmp["S21"] == df_tmp["S21"].max()]
                fpeak = tmp["Freq"].tolist()[0]
                gain = tmp["S21"].tolist()[0]

                # イメージ、ハーフイメージの算出
                fimg = fpeak - 100e6 # 50MHz x 2

                # イメージの計算
                if(fimg>100e6):
                    img = gain - df_tmp[df_tmp["Freq"]==fimg]["S21"].tolist()[0]
                else:
                    img = "NG"

                row = step3*(k+step2*(j+step3*i)) + l

                # 元データに挿入
                df_all.loc[row,"Val1"] = val0
                df_all.loc[row,"Val2"] = val1
                df_all.loc[row,"Val3"] = val2
                df_all.loc[row,"VTUNE12"] = T
                df_all.loc[row,"fpeak"] = fpeak
                df_all.loc[row,"gain"] = gain
                df_all.loc[row,"fimg"] = fimg
                df_all.loc[row,"img"] = img

In [86]:
# ファイル出力
df_all.to_csv(fname_out)